In [1]:
!pip install syllapy
!pip install ety

## Imports

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import nltk
from collections import Counter
import syllapy
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import ety
from tqdm import tqdm

import tensorflow as tf
from transformers import (
    BertTokenizer,
    TFBertModel,
    XLMRobertaTokenizer,
    TFXLMRobertaModel,
)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (
    Embedding,
    Bidirectional,
    LSTM,
    Dense,
    Dropout,
    Input,
    Flatten,
    concatenate,
    BatchNormalization,
)
from keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings("ignore")

tqdm.pandas()
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

c:\Users\focus\Develop\CU\DTSA-5511\final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\focus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Read and Process Data

Below, I pre-process the text data along with performing feature extraction.  The following code takes text input, performs various operations on it, and returns a set of linguistic statistics and processed data.

The following code:

- Converts the input text to lowercase
- Tokenizes the text into sentences using the Natural Language Toolkit (`nltk`), and each sentence is further tokenized into words. POS tags are assigned to each word.
- For each word in the text, the code does the following:
  - Determines the word's origin language (etymology).
  - Retrieves the full name of the POS tag from the `pos_mapping`.
  - Counts the number of syllables in the word.
  - Calculates the length of the word.
- Calculates various linguistic statistics, such as the mean syllable count, the number of sentences, the mean sentence length, the mean word length, and the total number of words.

In [3]:
df = pd.read_csv(
    "./kaggle/input/clear-corpus-6-01-clear-corpus-6-01/CLEAR Corpus 6.01 - CLEAR Corpus 6.01.csv"
)

In [4]:
pos_mapping = {
    "CC": "Coordinating Conjunction",
    "CD": "Cardinal Digit",
    "DT": "Determiner",
    "EX": "Existential There",
    "FW": "Foreign Word",
    "IN": "Preposition or Subordinating Conjunction",
    "JJ": "Adjective",
    "JJR": "Adjective, Comparative",
    "JJS": "Adjective, Superlative",
    "LS": "List Item Marker",
    "MD": "Modal",
    "NN": "Noun, Singular or Mass",
    "NNS": "Noun, Plural",
    "NNP": "Proper Noun, Singular",
    "NNPS": "Proper Noun, Plural",
    "PDT": "Predeterminer",
    "POS": "Possessive Ending",
    "PRP": "Personal Pronoun",
    "PRP$": "Possessive Pronoun",
    "RB": "Adverb",
    "RBR": "Adverb, Comparative",
    "RBS": "Adverb, Superlative",
    "RP": "Particle",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb, Base Form",
    "VBD": "Verb, Past Tense",
    "VBG": "Verb, Gerund or Present Participle",
    "VBN": "Verb, Past Participle",
    "VBP": "Verb, Non-3rd Person Singular Present",
    "VBZ": "Verb, 3rd Person Singular Present",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive Wh-pronoun",
    "WRB": "Wh-adverb",
}


def process_text(text):
    text = text.lower()

    word_origins = []
    word_pos = []
    syllable_counts = []
    sentence_lengths = []
    word_lengths = []

    sentences = sent_tokenize(text)

    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        pos_tags = nltk.pos_tag(tokens)
        sentence_lengths.append(len(pos_tags))
        for token, pos in pos_tags:
            origin = ety.origins(token)
            if origin:
                origin = origin[0].language.name
            else:
                origin = "unknown"
            word_origins.append(origin)
            full_pos_name = pos_mapping.get(pos, pos)
            word_pos.append(full_pos_name)
            syllables = syllapy.count(token)
            syllable_counts.append(syllables)
            word_lengths.append(len(token))

    processed_excerpt = text
    origin_counts = Counter(word_origins)
    pos_counts = Counter(word_pos)
    mean_syllable_count = np.mean(syllable_counts)
    num_sentences = len(sentences)
    mean_sentence_length = np.mean(sentence_lengths)
    num_words = np.sum(sentence_lengths)
    mean_word_length = np.mean(word_lengths)

    return (
        word_origins,
        origin_counts,
        word_pos,
        pos_counts,
        syllable_counts,
        mean_syllable_count,
        num_sentences,
        mean_sentence_length,
        mean_word_length,
        num_words,
        processed_excerpt,
    )

In [5]:
df[
    [
        "word_origins",
        "word_origin_counts",
        "pos",
        "pos_counts",
        "syllable_counts",
        "mean_syllable_count",
        "num_sentences",
        "mean_sentence_length",
        "mean_word_length",
        "num_words",
        "processed_excerpt",
    ]
] = df["Excerpt"].progress_apply(lambda x: pd.Series(process_text(x)))

100%|██████████| 4724/4724 [02:07<00:00, 37.05it/s]


## XGBoost Model

Below, I prepare text data for the machine learning model.

- **Extract Engineered Features:** I extract the sentence features, Parts of Speech counts, and language origin counts engineered above.
- **Train Model:** Train XGBoost model

In [6]:
sentence_features = df[
    [
        "mean_syllable_count",
        "num_sentences",
        "mean_sentence_length",
        "mean_word_length",
    ]
]

sentence_features = sentence_features.join(
    pd.DataFrame(df["pos_counts"].tolist()).fillna(0)
)

sentence_features = sentence_features.join(
    pd.DataFrame(df["word_origin_counts"].tolist()).fillna(0)
)

labels = np.array(df["BT Easiness"])

X_train, X_test, y_train, y_test = train_test_split(
    sentence_features, labels, test_size=0.2, random_state=42
)
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.6942651631843457
